In [1]:
%pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.25.1
    Uninstalling openai-1.25.1:
      Successfully uninstalled openai-1.25.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [12]:
import os
from openai import OpenAI

# OpenAI 객체 생성
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [101]:
# 어시스턴트 이름과 출력 파일 이름 설정
assistant_name = "파워포인트 생성기"
output_file_name = "../Assets/files/presentation7.pptx"

# 어시스턴트 지시사항 설정
assistant_instruction = r"{} 파일을 만들어줘. 너는 해당 주제의 전문가이자 PowerPoint 작성 전문가야. 대중을 상대로 발표할 수 있도록 콘텐츠 스타일이 아름답고 일목요연하게 정리해서 10개 이내로 슬라이드를 만들어줘.".format(output_file_name)

# 사용자 프롬프트 설정
prompt_user = "대한민국의 근로기준법(https://www.law.go.kr/LSW/lsLinkProc.do?lsNm=%EA%B7%BC%EB%A1%9C%EA%B8%B0%EC%A4%80%EB%B2%95&chrClsCd=010202&mode=20&ancYnChk=0)중 '근로시간과 휴식' 섹션을 참고하여 근로제 유형의 장점과 단점, 근무 규칙 그리고 어떤 경우에 사용하면 좋은지 프리젠테이션으로 만들어줘"

In [106]:
# 어시스턴트 생성 함수
def create_assistant(name, instruction):
    assistant = client.beta.assistants.create(
        name=name,
        instructions=instruction,
        tools=[
            { 
                "type": "code_interpreter",
            }
        ],
        model="gpt-4o",
    )
    return assistant
  

In [107]:
# 어시스턴트 생성
assistant = create_assistant(assistant_name, assistant_instruction)

In [108]:
# 대화 스레드 생성
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt_user)

In [109]:
# 실행 생성
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [110]:
import time

# 실행 상태 확인 및 완료 대기
timeout = 180  # 타임아웃 시간 (초)
interval_time = 5  # 체크 간격 (초)
time_taken = 0
while time_taken < timeout:
    run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id)

    if run.status == 'completed':
        break
    else:
        print(run.status)
        time.sleep(interval_time)
        time_taken += interval_time

in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


In [111]:
# 메시지 목록 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id)

# 파일 경로 추출
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id

# 파일 내용 가져오기
file_name = client.files.with_raw_response.content(file_path)

In [112]:
# 파일 저장
with open(output_file_name, "wb") as file:  
    file.write(file_name.content) 

In [113]:
# 파일 삭제
client.files.delete(file_path)

# 어시스턴트 삭제
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_b7gVTC3iHc4dmCTRebNxaQyQ', deleted=True, object='assistant.deleted')